In [6]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric
from torch_geometric.datasets import CoraFull, Planetoid, CitationFull
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric.nn as gnn 

from models import GAT
from utils import train_model, test_model

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
dataset = CitationFull(root='dataset/Cora', name='Cora', transform=NormalizeFeatures())

Processing...
Done!


In [4]:
data = dataset[0]
df = pd.DataFrame(data.x)
df['y'] = data.y
train, valid = train_test_split(df, stratify=df.y, test_size=0.33)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[train.index]=True

In [9]:
model=GAT(in_channels=dataset.num_features, hidden_channels=128, number_of_classes=dataset.num_classes, num_of_hidden_layers=2, device=device)
print(model.get_n_params())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

1141062


In [10]:
for epoch in range(500):
    loss = train_model(model, data, optimizer, criterion)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 4.2447
Epoch: 001, Loss: 4.2301
Epoch: 002, Loss: 4.1965
Epoch: 003, Loss: 4.1266
Epoch: 004, Loss: 4.0418
Epoch: 005, Loss: 4.0447
Epoch: 006, Loss: 4.0443
Epoch: 007, Loss: 4.0102
Epoch: 008, Loss: 4.0008
Epoch: 009, Loss: 4.0093
Epoch: 010, Loss: 4.0154
Epoch: 011, Loss: 4.0129
Epoch: 012, Loss: 4.0051
Epoch: 013, Loss: 3.9986
Epoch: 014, Loss: 3.9978
Epoch: 015, Loss: 3.9992
Epoch: 016, Loss: 3.9959
Epoch: 017, Loss: 3.9905
Epoch: 018, Loss: 3.9883
Epoch: 019, Loss: 3.9891
Epoch: 020, Loss: 3.9897
Epoch: 021, Loss: 3.9878
Epoch: 022, Loss: 3.9834
Epoch: 023, Loss: 3.9785
Epoch: 024, Loss: 3.9748
Epoch: 025, Loss: 3.9711
Epoch: 026, Loss: 3.9641
Epoch: 027, Loss: 3.9540
Epoch: 028, Loss: 3.9432
Epoch: 029, Loss: 3.9298
Epoch: 030, Loss: 3.9078
Epoch: 031, Loss: 3.8779
Epoch: 032, Loss: 3.8421
Epoch: 033, Loss: 3.7925
Epoch: 034, Loss: 3.7396
Epoch: 035, Loss: 3.6736
Epoch: 036, Loss: 3.6127
Epoch: 037, Loss: 3.5645
Epoch: 038, Loss: 3.5220
Epoch: 039, Loss: 3.4776


In [ ]:
test_acc, f1 = test_model(model, data)
print(f'Test Accuracy: {test_acc:.4f}')

              precision    recall  f1-score   support

           0       0.78      0.86      0.82       116
           1       0.91      0.69      0.79        72
           2       0.91      0.93      0.92       138
           3       0.85      0.87      0.86       270
           4       0.92      0.87      0.89       141
           5       0.83      0.82      0.82        98
           6       0.79      0.85      0.82        59

    accuracy                           0.86       894
   macro avg       0.86      0.84      0.85       894
weighted avg       0.86      0.86      0.86       894

Test Accuracy: 0.8568
